In [1]:
# src/forecast_prophet.py

import pandas as pd
from prophet import Prophet

def prophet_forecast_weekly(df_cat_weekly: pd.DataFrame, periods: int = 8) -> pd.DataFrame:
    """
    df_cat_weekly columns: week, review_count
    Returns forecast df with ds, yhat, yhat_lower, yhat_upper.
    """
    df = df_cat_weekly.copy()
    df = df.sort_values("week")

    # Prophet expects ds, y
    prophet_df = df.rename(columns={"week": "ds", "review_count": "y"})[["ds", "y"]]
    prophet_df["ds"] = pd.to_datetime(prophet_df["ds"])

    m = Prophet(weekly_seasonality=True, daily_seasonality=False, yearly_seasonality=False)
    m.fit(prophet_df)

    future = m.make_future_dataframe(periods=periods, freq="W-MON")
    forecast = m.predict(future)

    return forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]


def last8_vs_next8_summary(df_cat_weekly: pd.DataFrame, forecast_df: pd.DataFrame) -> dict:
    """
    Produces a clean summary you can paste into LinkedIn visuals:
    last8 avg actual vs next8 avg forecast + pct change.
    """
    df = df_cat_weekly.copy().sort_values("week")
    last8 = df.tail(8)["review_count"].mean()

    # get only future points (after last observed week)
    last_week = df["week"].max()
    future = forecast_df[forecast_df["ds"] > last_week].head(8)
    next8 = future["yhat"].mean()

    pct = ((next8 - last8) / last8) * 100 if last8 and last8 != 0 else None

    return {
        "last_observed_week": last_week,
        "last8_avg_actual": float(last8) if last8 == last8 else None,
        "next8_avg_forecast": float(next8) if next8 == next8 else None,
        "pct_change": float(pct) if pct == pct else None,
    }
